In [1]:
# imports
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import sys

# bring in the data and make some processing with the data
TEST_DIR = "C:/Users/Nancy/Documents/school/cnn/data/test"
TRAIN_DIR = "C:/Users/Nancy/Documents/school/cnn/data/train"
IMG_SIZE = 50 #resize the image to 50 by 50
LR = 1e-3 #learning rate is 0.001

# 
MODEL_NAME = 'skinColor-{}-{}.model'.format(LR, '2conv-basic-video')


In [2]:
# function to get the label of the image
def label_img(img):
    word_label = img.split('.')[0][:3]
    # with 5 people
    if   word_label == 'nan': return [1,0,0,0,0,0]
    elif word_label == 'emi': return [0,1,0,0,0,0]
    elif word_label == 'fel': return [0,0,1,0,0,0]
    elif word_label == 'ken': return [0,0,0,1,0,0]
    elif word_label == 'mar': return [0,0,0,0,1,0]
    elif word_label == 'mam': return [0,0,0,0,0,1]
    

In [3]:
# helper fxn
# create train data, labeled images
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])

    # shuffle the data
    shuffle(training_data)
    # save the training data numpy array
    # np.save('train_dataNew.npy', training_data)
    # return the traininng data
    return training_data


In [4]:
# competition text data with no label
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0] # getting the img
        #resizing the image
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    
    shuffle(testing_data)
    # save data
    # np.save('test_dataNew.npy', testing_data)
    return testing_data

In [5]:
# first run
train_data = create_train_data()
# if you already have train data
#train_data = np.load('train_data.npy')

100%|██████████████████████████████████████████████████████████████████████████████| 8512/8512 [03:32<00:00, 40.04it/s]


In [6]:
# using tflearn
#cnn

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression



convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
# convnet = fully_connected(convnet, 1, activation='softmax')

convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# All are labeled data
# separate the training and testing data for the accuracy of our models
train = train_data[:-500] # the train data is the last 500 sample data
test = train_data[-500:]
print("Total train data: "+str(len(train_data)))
print("Train data " + str(len(train)))
print("Test data "+ str(len(test)))

Total train data: 8512
Train data 8012
Test data 500


In [8]:
# Data getting fit
# the zero element was the image data, the pixel data
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 2) # featured set
Y = [i[1] for i in train] # Labels

# For testing accurancy
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 2) # featured set
test_y = [i[1] for i in test] # Labels

In [14]:
# reshape X and Y
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = np.array([i[1] for i in train]).reshape(-1,1)
# Y = Y.reshape(len(Y), -1)
Y = Y.reshape(len(Y), 1)
Y = [i[1] for i in train] # Labels

# Train the network
# model.fit({'input': X}, {'targets': Y}, n_epoch=6, validation_set=({'input': test_x}, {'targets': test_y}), 
#     snapshot_step=500, show_metric=True, run_id=MODEL_NAME) #run id is how we will find it in tensorboard

# model.fit( X, Y, n_epoch=5,
# validation_set=(test_x,test_y),
# snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

model.fit(X,Y, n_epoch=10, validation_set=(test_x,test_y), snapshot_step=500, show_metric=True, run_id=MODEL_NAME)


---------------------------------
Run id: skinColor-0.001-2conv-basic-video.model
Log directory: log/
---------------------------------
Training samples: 8012
Validation samples: 250
--


ValueError: Cannot feed value of shape (64, 6) for Tensor targets/Y:0, which has shape (?, 2)